In [1]:
from pymongo import MongoClient

In [7]:
DB_NAME = 'khadanga_crawl'
COLLECTION_NAME = 'blogs'

In [8]:
mongo_uri = "mongodb+srv://<usr>:<pwd>@cluster0.kf3n4.mongodb.net/{}?authSource=admin&ssl=true".format(DB_NAME)
conn = MongoClient(mongo_uri)

In [9]:
database = conn.get_default_database()

In [10]:
all_blogs = database[COLLECTION_NAME].find({"link":"https://arxiv.org/pdf/0704.0002"})

In [11]:
blogs = []
for blog in all_blogs:
    blogs.append(blog)

In [12]:
"Total count = {}".format(len(blogs))

'Total count = 1'

In [22]:
def body(b):
    return "{}".format(b['body'])
def title(b):
    return "{}".format(b['title'])
def abstract(x):
    return x['abstract']

In [28]:
import textacy
import textacy.ke
import spacy

In [41]:
from collections import namedtuple
Keywords = namedtuple('Keywords', ['title', 'body', 'abstract'])

def get_keywords(blog):
    tk, bk, ak = None,None,None
    if 'title' in blog:
        title_doc = textacy.make_spacy_doc(title(blog), lang='en')
        tk = textacy.ke.sgrank(title_doc, ngrams=(1,2,3), normalize="lower", include_pos=['NOUN', 'PROPN'])
    if 'body' in blog:
        body_doc = textacy.make_spacy_doc(body(blog), lang='en')
        bk = textacy.ke.sgrank(body_doc, ngrams=(1,2,3), normalize="lower", topn=5, include_pos=['NOUN', 'PROPN'])
    if 'abstract' in blog:
        abstract_doc = textacy.make_spacy_doc(abstract(blog), lang='en')
        ak = textacy.ke.sgrank(abstract_doc, ngrams=(1,2,3), normalize="lower", topn = 5,include_pos=['NOUN', 'PROPN'])        
    keywords = Keywords(tk,bk,ak)
    return keywords

In [195]:
import random
blog = random.choice(blogs)
keywords = get_keywords(blog)
print(title(blog))
print("{}\n {}".format("title", "\n ".join(map(lambda x: x[0],keywords.title))))
print("{}\n {}".format("body", "\n ".join(map(lambda x: x[0],keywords.body))))

DeepVariant: Highly Accurate Genomes With Deep Neural Networks
title
 neural networks
 genomes
 deepvariant
body
 reference genomes
 hts
 genomics
 image classification
 instruments
 brain
 source
 thegoogle
 reads
 data


In [196]:
print(body(blog))

Posted by Mark DePristo and Ryan Poplin, Google Brain Team(Crossposted on theGoogle Open Source Blog)Across many scientific disciplines, but in particular in the field of genomics, major breakthroughs have often resulted from new technologies. FromSanger sequencing, which made it possible to sequence the human genome, to themicroarray technologiesthat enabled the first large-scale genome-wide experiments, new instruments and tools have allowed us to look ever more deeply into the genome and apply the results broadly tohealth,agricultureandecology.One of the most transformative new technologies in genomics was high-throughput sequencing (HTS), which first became commercially available in the early 2000s. HTS allowed scientists and clinicians to produce sequencing data quickly, cheaply, and at scale. However, the output of HTS instruments is not the genome sequence for the individual being analyzed — for humans this is 3 billion paired bases (guanine, cytosine, adenine and thymine) organ

In [66]:
nlp = spacy.load('en')

In [67]:
nlp("this is a statement in english")

this is a statement in english

In [142]:
print(title(blogs[49]))

Google Handwriting Input in 82 languages on your Android mobile device


In [143]:
doc = textacy.make_spacy_doc(title(blogs[49]), lang='en')

In [144]:
doc._.preview

'Doc(11 tokens: "Google Handwriting Input in 82 languages on you...")'

In [120]:
textacy.ke.yake(
    doc, 
    normalize="lower", 
    ngrams=(1,2,3), 
    window_size=2, 
    include_pos=['NOUN', 'PROPN']
)

[('networks', 0.6392318376935061),
 ('quantum', 0.7493746385669857),
 ('neural', 0.7493746385669857),
 ('neural networks', 0.919474686762359),
 ('quantum neural', 1.2808260138096879),
 ('quantum neural networks', 1.809988089918808)]

In [121]:
textacy.ke.scake(
    doc, 
    normalize="lower", 
    include_pos=['NOUN', 'PROPN']
)

[('quantum neural networks', 2.1666666666666665)]

In [122]:
textacy.ke.sgrank(
    doc, 
    ngrams=(1,2,3), 
    normalize="lower", 
    include_pos=['NOUN', 'PROPN']
)

[]

In [123]:
textacy.ke.textrank(
    doc, 
    normalize="lower"
)

[('quantum neural networks', 0.8245616022774844)]

### Attach keywords to Paper

In [14]:
DB_NAME = 'TechVault'
COLLECTION_NAME = 'paper'

In [15]:
mongo_uri = "mongodb+srv://<usr>:<pwd>@cluster0.kf3n4.mongodb.net/{}?authSource=admin&ssl=true".format(DB_NAME)
conn = MongoClient(mongo_uri)
database = conn.get_default_database()

In [33]:
all_blogs = database[COLLECTION_NAME].find({"link":"https://arxiv.org/pdf/0704.0046"})

In [34]:
papers = []
for paper in all_blogs:
    papers.append(paper)

In [35]:
papers

[{'_id': ObjectId('60161bc2dc518a69be4aa445'),
  'date': '2009-11-13',
  'total_comments': 0,
  'link': 'https://arxiv.org/pdf/0704.0046',
  'contentId': 'edd3cd8287974dd885d8c4d70ab19df9',
  'abstract': '  In a quantum mechanical model, Diosi, Feldmann and Kosloff arrived at a\nconjecture stating that the limit of the entropy of certain mixtures is the\nrelative entropy as system size goes to infinity. The conjecture is proven in\nthis paper for density matrices. The first proof is analytic and uses the\nquantum law of large numbers. The second one clarifies the relation to channel\ncapacity per unit cost for classical-quantum channels. Both proofs lead to\ngeneralization of the conjecture.\n',
  'title': 'A limit relation for entropy and channel capacity per unit cost',
  'total_likes': 0,
  'authors': 'I. Csiszar, F. Hiai and D. Petz'}]

In [53]:
def clean_list_of_keywords(k):
    items = []
    if k.title:
        items += list(map(lambda x:x[0], k.title))
    if k.abstract:
        items += list(map(lambda x:x[0], k.abstract))
    return list(set(items))

In [54]:
clean_list_of_keywords(get_keywords(papers[0]))

['conjecture',
 'channel capacity',
 'unit cost',
 'system size',
 'limit relation',
 'density matrices',
 'quantum channels']

In [43]:
from pymongo import UpdateOne

In [ ]:
all_papers = database[COLLECTION_NAME].find().batch_size(50)
try:
    update_operations = []
    count = 0
    for paper in all_papers:
        operation = UpdateOne(
            {'_id': paper['_id']}, 
            {'$set': {'keywords':clean_list_of_keywords(get_keywords(paper))}}
        )
        update_operations.append(operation)
        if len(update_operations) == 500:
            # update here
            database[COLLECTION_NAME].bulk_write(update_operations, ordered=False)
            update_operations = []
            count += 500
            print(count)
except Exception as e:
    print(str(e))
finally:
    all_papers.close()